In [1]:
import pandas as pd
import re
from ast import literal_eval
import json

In [2]:
df = pd.read_csv('dataset/movies_metadata_ready_big2.csv', encoding='utf-8')
df = df.dropna()
df.shape

(20390, 14)

In [3]:
df_keywords = pd.read_csv('dataset/keywords.csv', encoding='utf-8')

In [4]:
df_keywords["keywords"] = df_keywords["keywords"].apply(literal_eval)
df_keywords["keywords"] = df_keywords["keywords"].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df_keywords = df_keywords.rename(columns={"keywords": "tags"})


In [5]:
df_keywords


,id,tags
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."
...,...,...
46414,439050,[tragic love]
46415,111109,"[artist, play, pinoy]"
46416,67758,[]
46417,227506,[]


In [6]:
df.dtypes


adult                 object
id                    object
original_language     object
original_title        object
overview              object
popularity            object
runtime              float64
tagline               object
title                 object
vote_average         float64
vote_count           float64
genres                object
release_year         float64
overview_keywords     object
dtype: object

In [7]:
df['id'] = df['id'].astype(int)

In [8]:
merged_df = pd.merge(df, df_keywords, on="id")


In [9]:
merged_df

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,release_year,overview_keywords,tags
0,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,104.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"['Adventure', 'Fantasy', 'Family']",1995.0,"['sibling', 'judy', 'peter', 'discover', 'ench...","[board game, disappearance, based on children'..."
1,False,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,101.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"['Romance', 'Comedy']",1995.0,"['family', 'wedding', 'reignite', 'ancient', '...","[fishing, best friend, duringcreditsstinger, o..."
2,False,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,127.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"['Comedy', 'Drama', 'Romance']",1995.0,"['cheat', 'mistreat', 'step', 'woman', 'hold',...","[based on novel, interracial relationship, sin..."
3,False,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,106.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,['Comedy'],1995.0,"['just', 'george', 'bank', 'recover', 'daughte...","[baby, midlife crisis, confidence, aging, daug..."
4,False,949,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,170.0,A Los Angeles Crime Saga,Heat,7.7,1886.0,"['Action', 'Crime', 'Drama', 'Thriller']",1995.0,"['obsessive', 'master', 'thief', 'neil', 'mcca...","[robbery, detective, bank, obsession, chase, s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20698,False,14885,en,Pooh's Heffalump Halloween Movie,"It's Halloween in the 100 Acre Wood, and Roo's...",2.568495,67.0,Celebrate Lumpy's First Halloween.,Pooh's Heffalump Halloween Movie,5.4,7.0,"['Animation', 'Family']",2005.0,"['halloween', 'acre', 'wood', 'roo', 'good', '...","[halloween, winnie the pooh]"
20699,False,420346,en,The Morning After,The Morning After is a feature film that consi...,0.139936,79.0,What happened last night?,The Morning After,4.0,2.0,"['Comedy', 'Drama']",2015.0,"['morning', 'feature', 'consist', 'vignette', ...",[]
20700,False,84419,en,House of Horrors,An unsuccessful sculptor saves a madman named ...,0.222814,65.0,Meet...The CREEPER!,House of Horrors,6.3,8.0,"['Horror', 'Mystery', 'Thriller']",1946.0,"['unsuccessful', 'sculptor', 'save', 'madman',...","[revenge, murder, serial killer, new york city..."
20701,False,289923,en,The Burkittsville 7,A film archivist revisits the story of Rustin ...,0.38645,30.0,"Do you know what happened 50 years before ""The...",The Burkittsville 7,7.0,1.0,['Horror'],2000.0,"['archivist', 'revisit', 'story', 'rustin', 'p...","[witch, mythology, legend, serial killer, mock..."


In [11]:
df_credits = pd.read_csv('dataset/credits.csv', encoding='utf-8')
df_credits["crew"] = df_credits["crew"].apply(literal_eval)
df_credits["directors"] = df_credits["crew"].apply(lambda x: [i['name'] for i in x if i["department"] == "Directing"] if isinstance(x, list) else [])
df_credits["cast"] = df_credits["cast"].apply(literal_eval)
df_credits["characters"] = df_credits["cast"].apply(lambda x: [i['character'] for i in x] if isinstance(x, list) else [])
df_credits["actors"] = df_credits["cast"].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [12]:
df_credits

,cast,crew,id,directors,characters,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[John Lasseter, Roman Figun, Desirée Mourad]","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,[Joe Johnston],"[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch],"[Max Goldman, John Gustafson, Ariel Gustafson,...","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker],"[Savannah 'Vannah' Jackson, Bernadine 'Bernie'...","[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,[Charles Shyer],"[George Banks, Nina Banks, Franck Eggelhoffer,...","[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,[Hamid Nematollah],"[, , ]","[Leila Hatami, Kourosh Tahami, Elham Korda]"
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,[Lav Diaz],"[Sister Angela, Homer, Crazy Woman/Virgin, Ama...","[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,[Mark L. Lester],"[Emily Shaw, Det. Mark Winston, Jayne Ferré, A...","[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,[Yakov Protazanov],"[, , , , ]","[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


In [13]:
df_credits.drop(columns=["crew", "cast"], inplace=True)

In [14]:
df_credits

,id,directors,characters,actors
0,862,"[John Lasseter, Roman Figun, Desirée Mourad]","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,8844,[Joe Johnston],"[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,15602,[Howard Deutch],"[Max Goldman, John Gustafson, Ariel Gustafson,...","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,31357,[Forest Whitaker],"[Savannah 'Vannah' Jackson, Bernadine 'Bernie'...","[Whitney Houston, Angela Bassett, Loretta Devi..."
4,11862,[Charles Shyer],"[George Banks, Nina Banks, Franck Eggelhoffer,...","[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...
45471,439050,[Hamid Nematollah],"[, , ]","[Leila Hatami, Kourosh Tahami, Elham Korda]"
45472,111109,[Lav Diaz],"[Sister Angela, Homer, Crazy Woman/Virgin, Ama...","[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45473,67758,[Mark L. Lester],"[Emily Shaw, Det. Mark Winston, Jayne Ferré, A...","[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
45474,227506,[Yakov Protazanov],"[, , , , ]","[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


In [15]:
df_ready = pd.merge(merged_df, df_credits, on="id")

In [16]:
df

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,release_year,overview_keywords
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,104.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"['Adventure', 'Fantasy', 'Family']",1995.0,"['sibling', 'judy', 'peter', 'discover', 'ench..."
2,False,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,101.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"['Romance', 'Comedy']",1995.0,"['family', 'wedding', 'reignite', 'ancient', '..."
3,False,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,127.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"['Comedy', 'Drama', 'Romance']",1995.0,"['cheat', 'mistreat', 'step', 'woman', 'hold',..."
4,False,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,106.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,['Comedy'],1995.0,"['just', 'george', 'bank', 'recover', 'daughte..."
5,False,949,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,170.0,A Los Angeles Crime Saga,Heat,7.7,1886.0,"['Action', 'Crime', 'Drama', 'Thriller']",1995.0,"['obsessive', 'master', 'thief', 'neil', 'mcca..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45449,False,14885,en,Pooh's Heffalump Halloween Movie,"It's Halloween in the 100 Acre Wood, and Roo's...",2.568495,67.0,Celebrate Lumpy's First Halloween.,Pooh's Heffalump Halloween Movie,5.4,7.0,"['Animation', 'Family']",2005.0,"['halloween', 'acre', 'wood', 'roo', 'good', '..."
45454,False,420346,en,The Morning After,The Morning After is a feature film that consi...,0.139936,79.0,What happened last night?,The Morning After,4.0,2.0,"['Comedy', 'Drama']",2015.0,"['morning', 'feature', 'consist', 'vignette', ..."
45456,False,84419,en,House of Horrors,An unsuccessful sculptor saves a madman named ...,0.222814,65.0,Meet...The CREEPER!,House of Horrors,6.3,8.0,"['Horror', 'Mystery', 'Thriller']",1946.0,"['unsuccessful', 'sculptor', 'save', 'madman',..."
45458,False,289923,en,The Burkittsville 7,A film archivist revisits the story of Rustin ...,0.38645,30.0,"Do you know what happened 50 years before ""The...",The Burkittsville 7,7.0,1.0,['Horror'],2000.0,"['archivist', 'revisit', 'story', 'rustin', 'p..."


In [17]:
df_ready.to_csv('dataset/movies_all_data.csv', index=False) 

In [20]:
df_ready['actors'] = df_ready['actors'].apply(lambda x: x[:3])

In [22]:
df_ready['characters'] = df_ready['characters'].apply(lambda x: x[:3])

In [23]:
df_ready.to_csv('dataset/movies_all_data2.csv', index=False) 